In [1]:
import pandas as pd
import numpy as np
from scipy.special import beta, btdtr

In [2]:
# Generate 100 Sample data
N = 100

# For each sample, 4 input values
X1 = np.random.binomial(1, 0.50, N)
X2 = np.random.binomial(1, 0.50, N)
X3 = np.random.binomial(1, 0.50, N)
X4 = np.random.binomial(1, 0.50, N)

# Output Y will depend only on X1 and X2
# parameters teta1 and teta2 are set to 0.75
P = 1 - (1 - 0.75*X1) * (1 - 0.75*X2)
Y = np.random.binomial(1, P)

In [3]:
data = pd.DataFrame({'X1': X1, 'X2': X2, 'X3': X3, 'X4': X4, 'Y': Y})

In [4]:
# find every posible combination of input sample values
comb = np.array(np.meshgrid([0,1],[0,1],[0,1],[0,1])).T.reshape(-1,4)
comb = [tuple(c) for c in comb]

In [5]:
# Use the samples to compute the probability of Y being 1, 
# for each of the posible input combinations
PY = {}
# Compute the frequency of each input combination
PX = {}
for c in comb:
    x1, x2, x3, x4 = c
    # Get samples that match current input combinacion
    filt = (data['X1'] == x1) & (data['X2'] == x2) & (data['X3'] == x3) & (data['X4'] == x4)
    df = data.loc[filt]
    # Compute PY and PX
    PY[c] = df['Y'].mean()
    PX[c] = len(df)/len(data)

PY = pd.Series(PY, name='PY')
PX = pd.Series(PX, name='PX')
pd.set_option('display.multi_sparse', False)

In [6]:
PY

0  0  0  0    0.000000
0  0  0  1    0.000000
0  0  1  0    0.000000
0  0  1  1    0.000000
0  1  0  0    0.875000
0  1  0  1    1.000000
0  1  1  0    0.833333
0  1  1  1    0.833333
1  0  0  0    0.750000
1  0  0  1    1.000000
1  0  1  0    0.666667
1  0  1  1    0.750000
1  1  0  0    1.000000
1  1  0  1    1.000000
1  1  1  0    1.000000
1  1  1  1    1.000000
Name: PY, dtype: float64

In [7]:
PX

0  0  0  0    0.09
0  0  0  1    0.07
0  0  1  0    0.02
0  0  1  1    0.11
0  1  0  0    0.08
0  1  0  1    0.03
0  1  1  0    0.06
0  1  1  1    0.06
1  0  0  0    0.04
1  0  0  1    0.05
1  0  1  0    0.06
1  0  1  1    0.04
1  1  0  0    0.08
1  1  0  1    0.09
1  1  1  0    0.05
1  1  1  1    0.07
Name: PX, dtype: float64

In [8]:
def rY(g, t):
    """Draws a random number from P(Y=1|g, t)"""
    if type(g) is not np.ndarray:
        g = np.array(g)
    if type(t) is not np.ndarray:
        t = np.array(t)

    Xi = np.random.choice(PX.index, p=PX)
    Xi = np.array(Xi)
    p = 1 - ((1 - t)**(g*Xi)).prod()
    return np.random.binomial(1, p)

In [9]:
def rT(i):
    """Draws a random number for Theta_i"""
    if g[i] == 1:
        # non-informative uniform distribution
        a = b = 1
    elif g[i] == 0:
        # 
        x = sT[i] / it
        y = sT2[i]/it - (sT[i]/it)**2
        a = x**2 * (1 - x) / y - x
        b = x * (1 - x)**2 / y - 1 + x

    return np.random.beta(a, b)

In [10]:
def pD():
    """Computes P(D| g, t)"""

    p = 0.
    for Xi, PXi in PX.iteritems():
        Xi = np.array(Xi)
        p += (1 - ((1 - t)**(g*Xi)).prod()) * PXi
    return p

In [12]:
def pT(i=None):

    if i is None:
        p = 1.
        for i in range(len(g)):
            p *= pT(i)
        return p
    else:

        if g[i] == 1:
            # non-informative uniform distribution
            a = b = 1
        elif g[i] == 0:
            x = sT[i] / it
            y = sT2[i]/it - (sT[i]/it)**2
            a = x**2 * (1 - x) / y - x
            b = x * (1 - x)**2 / y - 1 + x
            
        return t[i]**(a-1) * (1 - t[i])**(b-1) / beta(a, b)

In [13]:
# pilot run size
it = 1000
hT = np.random.beta(1, 1, (it, 4))
# Sum of theta values
sT = hT.sum(axis=0)
# Sum of squares for variance
sT2 = (hT**2).sum(axis=0)

In [14]:
# the variable selection vector (gamma)
g = np.array([1, 1, 1, 1])
# the parameter vector (theta)
t = hT[-1].copy()

In [17]:
for _ in range(10000):
    for j in range(4):
        t[j] = rT(j)
        g[j] = 1
        pd1, pt1, pg1 = pD(), pT(), 0.5
        g[j] = 0
        pd0, pt0, pg0 = pD(), pT(), 0.5
        if pd0 > 0:
            oj = pd1 * pt1 * pg1 / (pd0 * pt0 * pg0)
            p = oj / (1 + oj)
        else:
            p = 1
        g[j] = np.random.binomial(1, p)
    sT += t
    sT2 += t**2
    it += 1
    

In [20]:
it

11000

In [21]:
sT/it

array([ 0.50275859,  0.50427326,  0.50030454,  0.49790137])